In [ ]:
import hyperopt
import numpy as np
import simpy

from scipy.stats import expon, gamma, uniform

## Problem

Consider a take-out restaurant. It has a capacity of 30 people combined that can be in line, getting served, waiting for food after placing their order, or working in the restaurant.

Customers arrive according to a poisson process with intensity $\lambda_\text{arrival}$; if the store is at capacity, they leave.

There is a single line for the checkout counter(s). The amount of time it takes to ring someone up is exponential distribution with rate $\lambda_\text{checkout}$. The net revenue in USD of an order follows a gamma distribution with shape parameter $2$ and rate $\lambda_\text{rev}$.

There is also a single queue for the kitchen. There are two kinds of cooks, master cooks, and line cooks. An order goes to whomever is available first; with the order going to a line cook if no one is working. The amount of time it takes each to make an order is exponential with rates $\lambda_\text{master}$ and $\lambda_\text{line}$ respectively.

A checkout worker costs USD 15 per hour, a line cook costs USD 25 / hour, and a master cook costs 90 / hour.

Run the simulation, serving all customers that that arrive before one hour is (do not worry about paying overtime, to make this easy, assume they only get paid for one hour regardless of when they finish). Find the optimal combination of workers to maximize profits (total net revenue minus costs of workers). Use 50 function evaluations with hyperopt to do this. Each evaluations of the objective function should do 50 simulations of the system.

Use the following variance reduction techniques simultaneously:

- common random numbers
- conditioning on the number of people served
- two control variables (mean interarrival time, mean checkout time)

Fill out the following code skeleton, and do not change the names of the function or the arguments.

In [ ]:
T = 1
lam_arrival = 120
lam_checkout = 100
lam_line = 40
lam_master = 90
lam_cook = [lam_line, lam_master]
lam_rev = 0.5

costs = np.array([15, 25, 90])

def simulate(n):
    def arrivals():
        while True:
            U_st = uniform.rvs(size=2)
            dt = expon.rvs(scale=1/lam_arrival)
            cvs['dt'].append(dt)
            rev = 2 / lam_rev
            if env.now + dt < T:
                yield env.timeout(dt)
                if store.count < n_capacity:
                    store_rqt = store.request()
                    env.process(service(U_st, store_rqt))
                    revenue.append(rev)
            else:
                break
            
    def service(U, store_rqt):
        rqt_checkout = checkout.request()
        yield rqt_checkout
        st_checkout = expon.ppf(U[0], scale=1/lam_checkout)
        cvs['st'].append(st_checkout)
        yield env.timeout(st_checkout)
        checkout.release(rqt_checkout)
        rqt_cook0 = cook0.request()
        rqt_cook1 = cook1.request()
        results = yield simpy.AnyOf(env, [rqt_cook0, rqt_cook1])
        if rqt_cook0 in results:
            if rqt_cook1 in results:
                cook1.release(rqt_cook1)
            else:
                rqt_cook1.cancel()
            st_cook = expon.ppf(U[1], scale=1/lam_cook[0])
            yield env.timeout(st_cook)
            cook0.release(rqt_cook0)
        else:
            rqt_cook0.cancel()
            st_cook = expon.ppf(U[1], scale=1/lam_cook[1])
            yield env.timeout(st_cook)
            cook1.release(rqt_cook1)
        store.release(store_rqt)

    n_workers = np.sum(n)
    env = simpy.Environment()
    n_capacity = 30 - n_workers
    store = simpy.Resource(env=env, capacity=n_capacity)
    checkout = simpy.Resource(env=env, capacity=n[0])
    cook0 = simpy.Resource(env=env, capacity=n[1])
    cook1 = simpy.Resource(env=env, capacity=n[2])
    revenue = []
    cvs = {'dt': [], 'st': []}
    env.process(arrivals())
    env.run()
    total_revenue = np.sum(revenue) - np.sum(T * costs * n)
    dt = np.mean(cvs['dt']) - 1 / lam_arrival
    st = np.mean(cvs['st']) - 1 / lam_checkout
    return total_revenue, dt, st

def run_simulations(n):
    rev = np.zeros(50)
    dt = np.zeros(50)
    st = np.zeros(50)
    np.random.seed(42)
    for i in range(50):
        rev[i], dt[i], st[i] = simulate(n)
    cd = - np.cov(rev, dt)[0][1] / np.var(dt)
    cs = - np.cov(rev, st)[0][1] / np.var(st)
    rev_mod = rev + cd * dt + cs * st
    return rev_mod

def obj(n):
    rev = run_simulations(n)
    return (-1) * np.mean(rev)

In [ ]:
best_setup = hyperopt.fmin(fn=obj,
                           space=[hyperopt.hp.quniform('n{}'.format(i), 1, 10, 1) for i in range(3)],
                           algo=hyperopt.tpe.suggest,
                           max_evals=50)

In [ ]:
best_setup